## Load the Data

In [1]:
import sys
sys.path.append("/Users/anish.dhir/Documents/Research/gp_causal")

In [2]:
from pairs.generate_pairs import TubingenPairs

In [3]:
data_gen = TubingenPairs()

Load cause-effect pairs: 100%|██████████| 108/108 [00:02<00:00, 50.82it/s]


In [4]:
x, y, weight = [], [], []
for i in data_gen.pairs_generator():
    x.append(i[0])
    y.append(i[1])
    weight.append(i[2])

## Plot the data

Check for linearity etc.

In [ ]:
import matplotlib.pyplot as plt
import os

In [ ]:
savepath = "/Users/anish.dhir/Documents/Research/gp_causal/tuebingen_plots"
if not os.path.exists(savepath):
    os.makedirs(savepath)

In [ ]:
for i in range(len(x)):
    if x[i].shape[-1] == 1:
        plt.scatter(x[i], y[i])
        plt.title(f"Plot {i}")
        plt.savefig(f"{savepath}/Tuebingen: {i}")
        plt.clf()

## Try and train a GP

The basic premise here is to use the properties of the marginal likelihood to try and compare the the complexities of cause|effect vs. effect|cause

In [8]:
import gpytorch
from gpytorch.models import ApproximateGP
from gpytorch.variational import CholeskyVariationalDistribution
from gpytorch.variational import VariationalStrategy


class GPModel(ApproximateGP):
    def __init__(self, inducing_points):
        variational_distribution = CholeskyVariationalDistribution(inducing_points.size(0))
        variational_strategy = VariationalStrategy(self, inducing_points, variational_distribution, learn_inducing_locations=True)
        super(GPModel, self).__init__(variational_strategy)
        self.mean_module = gpytorch.means.ConstantMean()
        self.covar_module = gpytorch.kernels.ScaleKernel(gpytorch.kernels.RBFKernel())

    def forward(self, x):
        mean_x = self.mean_module(x)
        covar_x = self.covar_module(x)
        return gpytorch.distributions.MultivariateNormal(mean_x, covar_x)

In [9]:
import torch
from tqdm import trange
from sklearn.cluster import KMeans


# Find optimal model hyperparameters
def train(x, y, num_inducing):
    if len(x) < num_inducing + 1:
        inducing = x_train
    else:
        kmeans = KMeans(n_clusters=num_inducing).fit(x_train)
        inducing = kmeans.cluster_centers_
        inducing = torch.from_numpy(inducing.astype(float)).float()
    likelihood = gpytorch.likelihoods.GaussianLikelihood()
    model = GPModel(inducing)
    # Set initial lengthscale value
    init_lengthscale = torch.std(x) / 10
    model.covar_module.base_kernel.lengthscale = init_lengthscale
    model.train()
    likelihood.train()
    # Use the adam optimizer
    optimizer = torch.optim.Adam([
        {'params': model.parameters()},  # Includes GaussianLikelihood parameters
    ], lr=0.1)

    # "Loss" for GPs - the marginal log likelihood
    mll = gpytorch.mlls.ExactMarginalLogLikelihood(likelihood, model)

    training_iter = 10000
    loss_list = []
    t = trange(training_iter, desc="Running Model", leave=True, position=0)
    for i in t:
        # Zero gradients from previous iteration
        optimizer.zero_grad()
        # Output from model
        output = model(x)
        # Calc loss and backprop gradients
        loss = - mll(output, y[:, 0])
        loss.backward()
        loss_list.append(loss.item())
        optimizer.step()
        if i > 1000:
            # Need a convergence criteria
            if np.abs(np.mean(loss_list[-10:]) - np.mean(loss_list[-50:-10])) < np.std(loss_list[-50:-10]):
                break
        if i % 25 == 0:
            t.set_description(f"Loss: {loss}")
            t.refresh()
    return loss_list

In [10]:
from tqdm import tqdm_notebook as tqdm
from sklearn.preprocessing import StandardScaler
import numpy as np

np.random.seed(0)
torch.manual_seed(0)

correct_idx = []
wrong_idx = []
num_inducing = 2000

for i in tqdm(range(len(x)), desc="Epochs", leave=True, position=0):
    # Ignore the high dim
    if x[i].shape[-1] > 1:
        continue
    else:
        # Get data points
        x_train, y_train, weight_train = x[i], y[i], weight[i]
    # Make sure data is standardised 
    x_train = StandardScaler().fit_transform(x_train)
    y_train = StandardScaler().fit_transform(y_train)
    x_train, y_train = torch.from_numpy(x_train.astype(float)).float(), torch.from_numpy(y_train.astype(float)).float()
    # x -> y score
    loss_x_y = train(x=x_train, y=y_train, num_inducing=num_inducing)
    # x <- y score
    loss_y_x = train(x=y_train, y=x_train, num_inducing=num_inducing)
    if loss_x_y[-1] < loss_y_x[-1]:
        correct_idx.append(i)
    else:
        wrong_idx.append(i)

/Users/anish.dhir/.pyenv/versions/3.6.8/envs/gp_causal_3.6.8/lib/python3.6/site-packages/ipykernel_launcher.py:12: TqdmDeprecationWarning: This function will be removed in tqdm==5.0.0
Please use `tqdm.notebook.tqdm` instead of `tqdm.tqdm_notebook`
  if sys.path[0] == '':


Epochs:   0%|          | 0/108 [00:00<?, ?it/s]

Loss: 1.297794222831726:  10%|█         | 1001/10000 [03:14<29:05,  5.15it/s]


KeyboardInterrupt: 

In [ ]:
correct_weight = [weight[i] for i in correct_idx]
wrong_weight = [weight[i] for i in wrong_idx]

In [ ]:
accuracy = np.sum(correct_weight) / (np.sum(correct_weight) + np.sum(wrong_weight))

In [ ]:
accuracy

In [ ]:
correct_idx

In [ ]:
wrong_idx